In [1]:
# -*- coding: utf-8 -*-
__author__ = 'socurites@gmail.com'

"""
모델 평가하기
MNIST examples
"""

import tensorflow as tf
import tensorflow.contrib.slim as slim
#import c01_defining_models.s04_examples.mnist_deep_step_by_step_slim as mnist_model
#from utils.dataset_utils import load_batch
from datasets import mnist
#from datasets import tf_record_dataset

tf.logging.set_verbosity(tf.logging.INFO)











"""
MNIST network definition example by TF-Slim
"""

#import tensorflow as tf
#import tensorflow.contrib.slim as slim


def mnist_convnet(inputs, is_training=True):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                        activation_fn=tf.nn.relu,
                        weights_initializer=tf.truncated_normal_initializer(stddev=0.1)):
        with slim.arg_scope([slim.conv2d],
                            kernel_size=5):
            net = slim.conv2d(inputs=inputs, num_outputs=32, scope='conv1')
            net = slim.max_pool2d(inputs=net, kernel_size=[2, 2], scope='pool1')
            net = slim.conv2d(inputs=net, num_outputs=64, scope='conv2')
            net = slim.max_pool2d(inputs=net, kernel_size=[2, 2], scope='pool2')
            net = slim.flatten(inputs=net, scope='flatten')
            net = slim.fully_connected(inputs=net, num_outputs=1024, scope='fc3')
            net = slim.dropout(inputs=net, is_training=is_training, keep_prob=0.5, scope='dropout4')
            net = slim.fully_connected(inputs=net, num_outputs=10, activation_fn=None, scope='fc4')
    return net









from preprocessing import inception_preprocessing
#import tensorflow as tf

#from tensorflow.contrib import slim


def load_batch(dataset, batch_size=32, height=299, width=299, is_training=False):
    """Loads a single batch of data.
    
    Args:
      dataset: The dataset to load.
      batch_size: The number of images in the batch.
      height: The size of each image after preprocessing.
      width: The size of each image after preprocessing.
      is_training: Whether or not we're currently training or evaluating.
    
    Returns:
      images: A Tensor of size [batch_size, height, width, 3], image samples that have been preprocessed.
      images_raw: A Tensor of size [batch_size, height, width, 3], image samples that can be used for visualization.
      labels: A Tensor of size [batch_size], whose values range between 0 and dataset.num_classes.
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)
    image_raw, label = data_provider.get(['image', 'label'])
    
    # Preprocess image for usage by Inception.
    image = inception_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)
    
    # Preprocess the image for display purposes.
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)

    # Batch it up.
    images, images_raw, labels = tf.train.batch(
          [image, image_raw, label],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size)
    
    return images, labels, images_raw
#    return images, images_raw, labels




'''
# 평가 데이터 로드
'''
#mnist_tfrecord_dataset = tf_record_dataset.TFRecordDataset(
#mnist_tfrecord_dataset = mnist.TFRecordDataset(
#    tfrecord_dir='/home/itrocks/Git/Tensorflow/tf-slim-tutorial/raw_data/mnist/tfrecord',
#    tfrecord_dir='tmp/mnist_logs',
#    dataset_name='mnist',
#    num_classes=10)

# Selects the 'train' dataset.
#dataset = mnist_tfrecord_dataset.get_split(split_name='validation')
dataset = mnist.get_split(split_name='validation')
images, labels, _ = load_batch(dataset)

'''
# 모델 정의
'''
predictions = mnist_model.mnist_convnet(inputs=images, is_training=False)

'''
# 메트릭 정의
'''
predictions = tf.argmax(predictions, 1)
labels = tf.argmax(labels, 1)

# Define the metrics:
names_to_values, names_to_updates = slim.metrics.aggregate_metric_map({
    'eval/Accuracy': slim.metrics.streaming_accuracy(predictions, labels),
    # 'eval/Recall@5': slim.metrics.streaming_recall_at_k(predictions, labels, 5),
})

'''
# 평가하기
'''
# logging 경로 설정
log_dir = '/tmp/tfslim_model/'
eval_dir = '/tmp/tfslim_model-eval/'
if not tf.gfile.Exists(eval_dir):
    tf.gfile.MakeDirs(eval_dir)

if not tf.gfile.Exists(log_dir):
    raise Exception("trained check point does not exist at %s " % log_dir)
else:
    checkpoint_path = tf.train.latest_checkpoint(log_dir)


metric_values = slim.evaluation.evaluate_once(
    master='',
    checkpoint_path=checkpoint_path,
    logdir=eval_dir,
    num_evals=100,
    eval_op=names_to_updates.values(),
    final_op=names_to_values.values())

names_to_values = dict(zip(names_to_values.keys(), metric_values))
for name in names_to_values:
    print('%s: %f' % (name, names_to_values[name]))


AttributeError: module 'datasets.mnist' has no attribute 'TFRecordDataset'